# Multi-Model Market Forecasting & Risk Engine

A rigorous time-series forecasting pipeline comparing three modelling paradigms:

| Model | Type | Strength |
|-------|------|----------|
| **ARIMA-GARCH** | Econometric | Explicit volatility modelling, fat tails |
| **XGBoost** | Machine Learning | Non-linear feature interactions |
| **LSTM** | Deep Learning | Sequential dependency learning |

---

## Table of Contents

1. [Setup & Configuration](#setup)
2. [Data Ingestion](#data)
3. **Part A: ARIMA-GARCH** (Steps 1-10)
4. **Part B: XGBoost**
5. **Part C: LSTM**
6. **Part D: Consolidated Comparison**

<a id='setup'></a>
## 0. Setup & Configuration

In [ ]:
# === GLOBAL CONFIGURATION ===
TICKER = "SPY"              # Asset to analyze
START_DATE = "2018-01-01"   # Training start
END_DATE = "2024-12-31"     # Training end 
TEST_SIZE = 56              # Out-of-sample test days (~2.5 months)
RANDOM_SEED = 42
# ============================

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Reproducibility
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['font.size'] = 11

print("✓ All imports successful.")

<a id='data'></a>
## 1. Data Ingestion & Preprocessing

**Primary Source:** Alpaca Markets API  
**Fallback:** Yahoo Finance (yfinance)

In [ ]:
def fetch_data_alpaca(ticker, start, end):
    """Fetch data from Alpaca Markets API (primary source)."""
    try:
        from alpaca_trade_api import REST
        
        api_key = os.getenv('ALPACA_API_KEY')
        secret_key = os.getenv('ALPACA_SECRET_KEY')
        endpoint = os.getenv('ALPACA_ENDPOINT', 'https://paper-api.alpaca.markets')
        
        if not api_key or api_key == 'your_api_key_here':
            print("⚠ Alpaca API key not configured. Falling back to yfinance.")
            return None
        
        api = REST(api_key, secret_key, endpoint)
        
        # Fetch bars
        bars = api.get_bars(
            ticker,
            '1Day',
            start=start,
            end=end,
            adjustment='all'  # Split and dividend adjusted
        ).df
        
        if bars.empty:
            print(f"⚠ No data returned from Alpaca for {ticker}. Falling back to yfinance.")
            return None
            
        # Rename columns to match expected format
        bars = bars.reset_index()
        bars = bars.rename(columns={'timestamp': 'Date', 'close': 'Price'})
        bars = bars[['Date', 'Price']]
        bars['Date'] = pd.to_datetime(bars['Date']).dt.tz_localize(None)
        bars = bars.set_index('Date')
        
        print(f"✓ Data fetched from Alpaca: {len(bars)} observations")
        return bars
        
    except Exception as e:
        print(f"⚠ Alpaca fetch failed: {e}. Falling back to yfinance.")
        return None


def fetch_data_yfinance(ticker, start, end):
    """Fetch data from Yahoo Finance (fallback source)."""
    import yfinance as yf
    
    raw_df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=True)
    
    if raw_df.empty:
        raise ValueError(f"No data returned from yfinance for {ticker}")
    
    # Handle both single and multi-ticker column formats
    if isinstance(raw_df.columns, pd.MultiIndex):
        # Multi-ticker format: ('Close', 'SPY')
        df = raw_df['Close'].to_frame()
        df.columns = ['Price']
    else:
        # Single-ticker format: 'Close'
        df = raw_df[['Close']].copy()
        df.columns = ['Price']
    
    df.index.name = 'Date'
    print(f"✓ Data fetched from Yahoo Finance: {len(df)} observations")
    return df


def fetch_data(ticker, start, end):
    """Unified data fetcher: Alpaca first, yfinance fallback."""
    print(f"Fetching {ticker} from {start} to {end}...")
    
    # Try Alpaca first
    df = fetch_data_alpaca(ticker, start, end)
    
    # Fallback to yfinance
    if df is None:
        df = fetch_data_yfinance(ticker, start, end)
    
    return df

In [ ]:
# Fetch Data
df = fetch_data(TICKER, START_DATE, END_DATE)

# Compute Returns
df['Return'] = np.log(df['Price'] / df['Price'].shift(1))
df['PriceChange'] = df['Price'].diff()  # For ARIMA on levels
df.dropna(inplace=True)

print(f"\n✓ Final dataset: {len(df)} observations")
print(f"  Date Range: {df.index[0].date()} to {df.index[-1].date()}")
df.head()

In [ ]:
# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(df.index, df['Price'], 'b-', linewidth=1)
axes[0].set_title(f'{TICKER} Price Series')
axes[0].set_ylabel('Price ($)')

axes[1].plot(df.index, df['Return'], 'k-', alpha=0.7, linewidth=0.8)
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title(f'{TICKER} Log Returns')
axes[1].set_ylabel('Log Return')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

---
# Part A: ARIMA-GARCH

**Methodology:** 10-step diagnostic pipeline for rigorous econometric modelling.

### A.1: Stationarity Test (ADF)

In [ ]:
def adf_test(series, name='Series'):
    result = adfuller(series.dropna(), autolag='AIC')
    print(f"ADF Test: {name}")
    print(f"  Statistic: {result[0]:.4f}")
    print(f"  p-value:   {result[1]:.4f}")
    print(f"  Lags Used: {result[2]}")
    print(f"  Stationary: {'Yes ✓' if result[1] < 0.05 else 'No ✗'}")
    return result[1] < 0.05

print("=" * 40)
adf_test(df['Price'], 'Raw Price')
print("=" * 40)
adf_test(df['PriceChange'], 'Price Change (d=1)')
print("=" * 40)
is_stationary = adf_test(df['Return'], 'Log Returns')

### A.2: Mean Model Selection (AR Order)

In [ ]:
# Use Returns (stationary series) for GARCH
series = df['Return'].dropna()

# Test for significant mean (intercept)
t_stat, p_val = stats.ttest_1samp(series, 0)
USE_INTERCEPT = p_val < 0.05
print(f"T-test for mean≠0: p={p_val:.4f} → {'Include' if USE_INTERCEPT else 'Exclude'} intercept")

# AR Order Selection
ar_aic = {}
trend = 'c' if USE_INTERCEPT else 'n'

for p in range(1, 6):
    try:
        fit = ARIMA(series, order=(p, 0, 0), trend=trend).fit()
        ar_aic[p] = fit.aic
    except:
        ar_aic[p] = np.inf

AR_ORDER = min(ar_aic, key=ar_aic.get)
print(f"\nAR Order Selection (AIC):")
for p, aic in ar_aic.items():
    marker = " ← BEST" if p == AR_ORDER else ""
    print(f"  AR({p}): AIC = {aic:.2f}{marker}")

### A.3: GARCH Distribution Selection

In [ ]:
# Scale returns (GARCH numerical stability)
scaled_series = series * 100

distributions = ['normal', 't', 'skewt']
garch_results = {}

print("GARCH(1,1) Distribution Selection:")
for dist in distributions:
    try:
        am = arch_model(scaled_series, mean='AR', lags=AR_ORDER, vol='Garch', p=1, q=1, dist=dist)
        fit = am.fit(disp='off')
        garch_results[dist] = {'fit': fit, 'aic': fit.aic}
        print(f"  {dist:8s}: AIC = {fit.aic:.2f}")
    except Exception as e:
        print(f"  {dist:8s}: FAILED ({e})")

BEST_DIST = min(garch_results, key=lambda x: garch_results[x]['aic'])
best_garch_fit = garch_results[BEST_DIST]['fit']
print(f"\n✓ Selected: AR({AR_ORDER})-GARCH(1,1) with {BEST_DIST} innovations")

In [ ]:
print(best_garch_fit.summary())

### A.4: In-Sample Volatility Bands

In [ ]:
cond_vol = best_garch_fit.conditional_volatility / 100
aligned_series = series.iloc[-len(cond_vol):]

inside = (aligned_series.values > -2*cond_vol.values) & (aligned_series.values < 2*cond_vol.values)
coverage = inside.mean()

plt.figure(figsize=(14, 6))
plt.plot(aligned_series.index, aligned_series.values, 'k-', alpha=0.7, label='Returns')
plt.fill_between(aligned_series.index, -2*cond_vol.values, 2*cond_vol.values, 
                 alpha=0.3, color='orange', label=f'±2σ (Coverage: {coverage:.1%})')
plt.axhline(0, color='red', linestyle='--', alpha=0.3)
plt.title('GARCH In-Sample Volatility Bands')
plt.legend()
plt.show()

### A.5: Walk-Forward Backtest

In [ ]:
train_end = len(series) - TEST_SIZE

arima_forecasts = []
garch_forecasts = []
garch_sigmas = []
actuals = []
dates = []

for i in tqdm(range(TEST_SIZE), desc="ARIMA-GARCH Backtest"):
    train = series.iloc[:train_end + i]
    actual = series.iloc[train_end + i]
    date = series.index[train_end + i]
    
    # ARIMA baseline
    try:
        ar_fit = ARIMA(train, order=(AR_ORDER, 0, 0), trend=trend).fit()
        ar_fc = ar_fit.forecast(steps=1).iloc[0]
    except:
        ar_fc = 0.0
    
    # AR-GARCH
    try:
        am = arch_model(train * 100, mean='AR', lags=AR_ORDER, vol='Garch', p=1, q=1, dist=BEST_DIST)
        g_fit = am.fit(disp='off', show_warning=False)
        fc = g_fit.forecast(horizon=1, reindex=False)
        mean_fc = fc.mean.iloc[-1, 0] / 100
        vol_fc = np.sqrt(fc.variance.iloc[-1, 0]) / 100
    except:
        mean_fc, vol_fc = 0.0, 0.01
    
    arima_forecasts.append(ar_fc)
    garch_forecasts.append(mean_fc)
    garch_sigmas.append(vol_fc)
    actuals.append(actual)
    dates.append(date)

In [ ]:
# Store ARIMA-GARCH results
arima_results = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'ARIMA_Pred': arima_forecasts,
    'GARCH_Pred': garch_forecasts,
    'GARCH_Sigma': garch_sigmas
}).set_index('Date')

arima_mse = mean_squared_error(arima_results['Actual'], arima_results['ARIMA_Pred'])
garch_mse = mean_squared_error(arima_results['Actual'], arima_results['GARCH_Pred'])

print(f"MSE (ARIMA only):  {arima_mse:.8f}")
print(f"MSE (AR-GARCH):    {garch_mse:.8f}")

---
# Part B: XGBoost

**Methodology:** Gradient Boosted Trees on lagged features with strict time-series validation.

### B.1: Feature Engineering

In [ ]:
# Create lagged features
N_LAGS = 5
VOL_WINDOW = 21

feature_df = df[['Return']].copy()

for lag in range(1, N_LAGS + 1):
    feature_df[f'lag_{lag}'] = feature_df['Return'].shift(lag)

feature_df['rolling_vol'] = feature_df['Return'].shift(1).rolling(VOL_WINDOW).std()
feature_df['ewma_vol'] = feature_df['Return'].shift(1).ewm(span=VOL_WINDOW).std()

feature_df.dropna(inplace=True)

# Define X and y
FEATURE_COLS = [c for c in feature_df.columns if c != 'Return']
X_full = feature_df[FEATURE_COLS]
y_full = feature_df['Return']

print(f"Features: {FEATURE_COLS}")
print(f"Total samples: {len(X_full)}")

### B.2: Walk-Forward Backtest

In [ ]:
train_end = len(X_full) - TEST_SIZE

xgb_forecasts = []
xgb_actuals = []
xgb_dates = []

XGB_PARAMS = {
    'n_estimators': 100,
    'max_depth': 3,
    'learning_rate': 0.05,
    'objective': 'reg:squarederror',
    'verbosity': 0,
    'n_jobs': -1,
    'random_state': RANDOM_SEED
}

for i in tqdm(range(TEST_SIZE), desc="XGBoost Backtest"):
    # Split
    X_train = X_full.iloc[:train_end + i]
    y_train = y_full.iloc[:train_end + i]
    X_test = X_full.iloc[[train_end + i]]
    y_test = y_full.iloc[train_end + i]
    
    # Scale (fit on train only!)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train
    model = xgb.XGBRegressor(**XGB_PARAMS)
    model.fit(X_train_scaled, y_train)
    
    # Predict
    pred = model.predict(X_test_scaled)[0]
    
    xgb_forecasts.append(pred)
    xgb_actuals.append(y_test)
    xgb_dates.append(X_full.index[train_end + i])

In [ ]:
xgb_results = pd.DataFrame({
    'Date': xgb_dates,
    'Actual': xgb_actuals,
    'XGB_Pred': xgb_forecasts
}).set_index('Date')

xgb_mse = mean_squared_error(xgb_results['Actual'], xgb_results['XGB_Pred'])
print(f"XGBoost MSE: {xgb_mse:.8f}")

---
# Part C: LSTM

**Methodology:** Recurrent Neural Network with sequence-based input.

### C.1: LSTM Architecture

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size=32, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Hyperparameters
LSTM_CONFIG = {
    'hidden_size': 32,
    'num_layers': 1,
    'epochs': 30,
    'lr': 0.01,
    'seq_length': 10
}

print(f"LSTM Config: {LSTM_CONFIG}")

### C.2: Sequence Creation

In [ ]:
def create_sequences(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

### C.3: Walk-Forward Backtest

In [ ]:
lstm_forecasts = []
lstm_actuals = []
lstm_dates = []

SEQ_LEN = LSTM_CONFIG['seq_length']
train_end = len(X_full) - TEST_SIZE

for i in tqdm(range(TEST_SIZE), desc="LSTM Backtest"):
    # Expanding window
    X_train = X_full.iloc[:train_end + i].values
    y_train = y_full.iloc[:train_end + i].values
    
    # Scale
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    
    # Create sequences
    X_seq, y_seq = create_sequences(X_train_scaled, y_train, SEQ_LEN)
    
    if len(X_seq) < 10:
        lstm_forecasts.append(0.0)
        lstm_actuals.append(y_full.iloc[train_end + i])
        lstm_dates.append(X_full.index[train_end + i])
        continue
    
    # Tensors
    X_tensor = torch.tensor(X_seq, dtype=torch.float32)
    y_tensor = torch.tensor(y_seq, dtype=torch.float32).unsqueeze(1)
    
    # Model
    model = LSTMNet(X_train.shape[1], LSTM_CONFIG['hidden_size'], LSTM_CONFIG['num_layers'])
    optimizer = optim.Adam(model.parameters(), lr=LSTM_CONFIG['lr'])
    criterion = nn.MSELoss()
    
    # Train
    model.train()
    for _ in range(LSTM_CONFIG['epochs']):
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = criterion(out, y_tensor)
        loss.backward()
        optimizer.step()
    
    # Predict: use last SEQ_LEN rows
    X_test_raw = X_full.iloc[train_end + i - SEQ_LEN + 1 : train_end + i + 1].values
    X_test_scaled = scaler_X.transform(X_test_raw)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        pred = model(X_test_tensor).item()
    
    lstm_forecasts.append(pred)
    lstm_actuals.append(y_full.iloc[train_end + i])
    lstm_dates.append(X_full.index[train_end + i])

In [ ]:
lstm_results = pd.DataFrame({
    'Date': lstm_dates,
    'Actual': lstm_actuals,
    'LSTM_Pred': lstm_forecasts
}).set_index('Date')

lstm_mse = mean_squared_error(lstm_results['Actual'], lstm_results['LSTM_Pred'])
print(f"LSTM MSE: {lstm_mse:.8f}")

---
# Part D: Consolidated Comparison

**Objective:** Compare all three models on the same out-of-sample test set.

In [ ]:
# Merge all results
comparison = arima_results[['Actual', 'GARCH_Pred']].copy()
comparison['XGB_Pred'] = xgb_results['XGB_Pred']
comparison['LSTM_Pred'] = lstm_results['LSTM_Pred']

comparison.head()

In [ ]:
# Compute metrics for each model
def compute_metrics(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    # Direction accuracy
    dir_acc = np.mean(np.sign(actual) == np.sign(predicted))
    return {'MSE': mse, 'MAE': mae, 'Direction Acc': dir_acc}

metrics = {
    'ARIMA-GARCH': compute_metrics(comparison['Actual'], comparison['GARCH_Pred']),
    'XGBoost': compute_metrics(comparison['Actual'], comparison['XGB_Pred']),
    'LSTM': compute_metrics(comparison['Actual'], comparison['LSTM_Pred'])
}

metrics_df = pd.DataFrame(metrics).T
metrics_df = metrics_df.sort_values('MSE')

print("=" * 50)
print(f"MODEL COMPARISON: {TICKER} ({TEST_SIZE}-day backtest)")
print("=" * 50)
display(metrics_df.round(6))
print(f"\n✓ BEST MODEL (by MSE): {metrics_df.index[0]}")

In [ ]:
# Visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

models = ['GARCH_Pred', 'XGB_Pred', 'LSTM_Pred']
titles = ['ARIMA-GARCH', 'XGBoost', 'LSTM']
colors = ['red', 'green', 'purple']

for ax, model, title, color in zip(axes, models, titles, colors):
    ax.plot(comparison.index, comparison['Actual'], 'k-', label='Actual', alpha=0.8, linewidth=1.5)
    ax.plot(comparison.index, comparison[model], color=color, linestyle='--', label=f'{title} Prediction', alpha=0.8)
    ax.axhline(0, color='gray', linestyle=':', alpha=0.3)
    ax.set_title(f'{title} Forecast vs Actual')
    ax.set_ylabel('Return')
    ax.legend(loc='upper right')

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for i, metric in enumerate(['MSE', 'MAE', 'Direction Acc']):
    values = metrics_df[metric].values
    colors = ['green' if v == values.min() else 'steelblue' for v in values] if metric != 'Direction Acc' else \
             ['green' if v == values.max() else 'steelblue' for v in values]
    
    axes[i].bar(metrics_df.index, values, color=colors)
    axes[i].set_title(metric)
    axes[i].set_ylabel(metric)
    for j, v in enumerate(values):
        axes[i].text(j, v, f'{v:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---
## Final Summary

In [ ]:
print("=" * 60)
print(f"FINAL REPORT: {TICKER}")
print("=" * 60)
print(f"\nData: {START_DATE} to {END_DATE} ({len(df)} obs)")
print(f"Backtest: Last {TEST_SIZE} days (walk-forward)")
print(f"\n{'Model':<15} {'MSE':<12} {'MAE':<12} {'Dir Acc':<12}")
print("-" * 51)
for model, row in metrics_df.iterrows():
    print(f"{model:<15} {row['MSE']:<12.6f} {row['MAE']:<12.6f} {row['Direction Acc']:<12.2%}")
print("-" * 51)
print(f"\n✓ Winner (MSE): {metrics_df.index[0]}")
print(f"✓ Winner (Direction): {metrics_df.sort_values('Direction Acc', ascending=False).index[0]}")